# We will collect text data using Twitter API.

In [2]:
import numpy as np
import pandas as pd
import tweepy  #to communicate with twitter API
import json
from tweepy import OAuthHandler

In [3]:
consumer_key = "CZczqRkYqz6o0utu39sg09XbA"
consumer_secret = "84FW99EM5ae0KwwQbobKRjA6SB2gDGX2ylA7ECBhz4IDAhhCpS"
access_token = "1321748368893173760-PYLWKOs3WxpyzuZuim4ASQD10QEpe4"
access_token_secret = "1WE05e2fqGbyLm9eeGG0SXuNAJGtnYcsAbg0IwoHosQ4Q"

In [4]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )  #to scrap the data from twitter
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [24]:
query = "Justice for Sushanth"

In [25]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )  #donot include the retweets, as sentiments will be duplicated
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x000001C679D7BC88>, _json={'created_at': 'Sun Nov 01 08:38:03 +0000 2020', 'id': 1322820184705388544, 'id_str': '1322820184705388544', 'full_text': "@AnamikaRashmi @narendramodi I can understand you want justice for sushanth but making it political is a completely different thing, it's not right.", 'truncated': False, 'display_text_range': [29, 148], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'AnamikaRashmi', 'name': '@n@mik@ || 302 RIGHT NOW', 'id': 312141092, 'id_str': '312141092', 'indices': [0, 14]}, {'screen_name': 'narendramodi', 'name': 'Narendra Modi', 'id': 18839785, 'id_str': '18839785', 'indices': [15, 28]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': 1322769065778307072, 'in_reply_to_status_id_str': '1322769065778307072', 'in_reply_to_user_id': 

In [58]:
#Get the tweets and some Attributes
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [55]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():  #count is returning the number of results in 1 page i.e, take 100 tweets from 1 page
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

#we are running this 1000 times, so that we get 1000 records in excel

In [59]:
stream(data=["Justice for Sushanth"] , file_name='my_tweets')

In [29]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,@AnamikaRashmi @narendramodi I can understand ...,IT Cell Spotter- गुप्तचर🕵️,1686,43,Somewhere,False,0,0,2020-11-01 08:38:03
1,We are eagerly waiting for justice for Sushant...,Rekha.Geetha,1195,6,,False,0,0,2020-11-01 07:48:46
2,RT @Mansa95335693: @AnimeFanIndia2 @iamsmritiy...,SSR Justice Warriors ✊👊⚔️⚔️,11047,297,,False,0,2,2020-11-01 06:20:20
3,Justice for sushanth #SSRJusticeDelayed,venu yadav,40,12,,False,0,0,2020-11-01 06:19:14
4,@pie_lioness @aaryaiyer99 I will celebrate diw...,justice for sushanth ✊✊✊,3535,50,,False,0,0,2020-11-01 06:05:10


In [11]:
#pip install textblob
from textblob import TextBlob

In [30]:
#cleaning the tweets
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [31]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [32]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [33]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,@AnamikaRashmi @narendramodi I can understand ...,IT Cell Spotter- गुप्तचर🕵️,1686,43,Somewhere,False,0,0,2020-11-01 08:38:03,I can understand you want justice for sushanth...
1,We are eagerly waiting for justice for Sushant...,Rekha.Geetha,1195,6,,False,0,0,2020-11-01 07:48:46,We are eagerly waiting for justice for Sushant...
2,RT @Mansa95335693: @AnimeFanIndia2 @iamsmritiy...,SSR Justice Warriors ✊👊⚔️⚔️,11047,297,,False,0,2,2020-11-01 06:20:20,RT If u true SSR fan don t follow any bollywoo...
3,Justice for sushanth #SSRJusticeDelayed,venu yadav,40,12,,False,0,0,2020-11-01 06:19:14,Justice for sushanth SSRJusticeDelayed
4,@pie_lioness @aaryaiyer99 I will celebrate diw...,justice for sushanth ✊✊✊,3535,50,,False,0,0,2020-11-01 06:05:10,lioness I will celebrate diwali only for susha...


In [34]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,@AnamikaRashmi @narendramodi I can understand ...,IT Cell Spotter- गुप्तचर🕵️,1686,43,Somewhere,False,0,0,2020-11-01 08:38:03,I can understand you want justice for sushanth...,Positive
1,We are eagerly waiting for justice for Sushant...,Rekha.Geetha,1195,6,,False,0,0,2020-11-01 07:48:46,We are eagerly waiting for justice for Sushant...,Neutral
2,RT @Mansa95335693: @AnimeFanIndia2 @iamsmritiy...,SSR Justice Warriors ✊👊⚔️⚔️,11047,297,,False,0,2,2020-11-01 06:20:20,RT If u true SSR fan don t follow any bollywoo...,Positive
3,Justice for sushanth #SSRJusticeDelayed,venu yadav,40,12,,False,0,0,2020-11-01 06:19:14,Justice for sushanth SSRJusticeDelayed,Neutral
4,@pie_lioness @aaryaiyer99 I will celebrate diw...,justice for sushanth ✊✊✊,3535,50,,False,0,0,2020-11-01 06:05:10,lioness I will celebrate diwali only for susha...,Positive


In [57]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 We are eagerly waiting for justice for Sushanth Singh Rajput p. CBI please give us justice we are begging you 🙏 🙏🙏🙏… https://t.co/Q48HxfGYfn

Clean tweet:
 We are eagerly waiting for justice for Sushanth Singh Rajput p CBI please give us justice we are begging you

Sentiment of the tweet:
 Neutral


In [44]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @IreshaMadhusha9: We talked for SSR, not because we're stuck or we haven't moved on.

We talk for SSR because we love SSR and he was, is…

Clean tweet:
 RT We talked for SSR not because we re stuck or we haven t moved on We talk for SSR because we love SSR and he was is

Sentiment of the tweet:
 Positive


In [49]:
n = 130
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @SSRFAN21905120: #PMSpeakUpOnSSRCase 
Sir India wants to know Who Killed Sushanth ?
What happened with Disha and Minor Girl?
why are you…

Clean tweet:
 RT PMSpeakUpOnSSRCase Sir India wants to know Who Killed Sushanth What happened with Disha and Minor Girl why are you

Sentiment of the tweet:
 Negative


In [53]:
df[df.Sentiment == 'Positive'].shape[0]

85

In [51]:
df[df.Sentiment == 'Neutral'].shape[0]

82

In [52]:
df[df.Sentiment == 'Negative'].shape[0]

30